# QC Machine Learning Workshop Notebook
[Github Repository](https://github.com/ucgmsim/qc_ml_workshop_2024)

## Content
This notebook is a general introduction to machine learning. It covers the following topics:
- **Supervised Machine Learning Overview**
- **Data Preparation**
- **Model Fitting**
- **Model Evaluation & Hyperparameters**

#### Notebook Instructions
Run cells containing code by clicking on them and hitting **Ctrl+Enter** or by Cell>Run Cells in the drop-down menu.

#### Binder Timeout
Binder has a timeout of 10 minutes. If you are inactive for more than 10 minutes, the session will be terminated. You can restart the session by clicking on the Binder link again. Changes you made to the notebook will be lost.

### Figure Credit
All figures are from the book "Sebastian Raschka, Yuxi (Hayden) Liu, and Vahid Mirjalili. Machine Learning with PyTorch and Scikit-Learn. Packt Publishing, 2022."  
 

## Machine Learning Overview


### Types of Machine Learning
<img src="resources/imgs/ml_types.png" width="1000"/>

**Only looking at supervised ML in this workshop.**

### Typical supervised ML workflow
<img src="resources/imgs/ml_workflow.png" width="1000"/>

### Classification: Predicting a class label  

<img src="resources/imgs/ml_classification.png" width="1000">

### Regression: Predicting a continuous value  

<img src="resources/imgs/ml_regression.png" width="1000">

## Python Packages
- **Pandas**: Data manipulation, [website](https://pandas.pydata.org/), [user guide](https://pandas.pydata.org/docs/user_guide/index.html)
- **Numpy**: Numerical operations, [website](https://numpy.org/), [user guide](https://numpy.org/doc/stable/user/index.html)
- **Scikit-learn**: Machine learning models, [website](https://scikit-learn.org/stable/index.html), [user guide](https://scikit-learn.org/stable/user_guide.html)
- **Matplotlib**: Plotting, [website](https://matplotlib.org/), [user guide](https://matplotlib.org/stable/users/index)

## Resources for further learning

#### Free, Online
- [Kaggle "Courses"](https://www.kaggle.com/learn)
- [Scikit-learn User Guide](https://scikit-learn.org/stable/user_guide.html)

#### Books
- Machine Learning with Pytorch and Scitkit-learn, Sebastian Raschka
- Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow, Aurélien Géron

## Problem Setup
Before starting the process of developing a ML model it is important to consider the following:
- **Problem Definition**: What are you trying to predict?
- **Problem Type**: Classification or Regression
- **Data Availability**: Do you have enough data and is it of sufficient quality?
- **Evaluation Metric**: How will you evaluate the model performance? For a list of evaluation metrics see the [Scikit-learn documentation](https://scikit-learn.org/stable/modules/model_evaluation.html)

### Workshop Problem
- **Problem Definition**: Predict the weather pattern based on temperature, humidity and location type
- **Problem Type**: Classification
- **Data Availability**: Sufficient data available (see below)
- **Evaluation Metric**: Accuracy


In [ ]:
import notebook_code as nc
df = nc.load_weather_data_pre(with_missing_values=True)
df.head()

### ML Terminology
**Sample**: A single instance of data  
**Feature**: An input variable used to make a prediction  
**Label/Target**: The output variable we are trying to predict  
**Model**: The algorithm used to make predictions  
**Training**: The process of learning the model parameters from the data

## Data Preparation

### Data Curation
- Data collection
- Data cleaning, e.g. handling of missing data

### Data Exploration
- Understanding data distributions
- Identifying relationships between features and the target variable
- Identifying anomalies and outliers
- Identifying any potential biases in the data, e.g. imbalanced dataset
- Identifying and selecting features

**Example: Credit Card Fraud Label Imbalance**  
<img src="resources/imgs/credit_card_fraud_imbalance.png">  
Figure from this [Kaggle notebook](https://www.kaggle.com/code/janiobachmann/credit-fraud-dealing-with-imbalanced-datasets)

### Data Preprocessing
- Feature scaling
- Encoding of categorial features
- Encoding of target variable (if needed)

Additional resources:
- [Scikit-learn preprocessing](https://scikit-learn.org/stable/modules/preprocessing.html)

In [ ]:
# Load data
import notebook_code as nc
df = nc.load_weather_data_pre(with_missing_values=True)

# Get size of the dataset
print(df.shape)

In [ ]:
# Display the first few rows of the dataset
df.head()

In [ ]:
# Get number of samples for each class
df.groupby("target").size()

### Missing Data

How to handle:
- Remove rows or features with missing data
- Impute missing data, i.e. use interpolation techniques to infer missing values from the other samples, for example using the mean or median of the feature.

Handling of missing data is dataset/problem specific!    

In [ ]:
# Check for missing data
# Note: This isn't always as easy, sometimes missing data is encoded as multiple specific values, e.g. N/A, -999, NA, etc.
df.loc[df.isna().any(axis=1)].head(10)

In [ ]:
df.isna().sum(axis=0)

In [ ]:
# Drop rows with missing data
print(f"Number of samples before dropping: {len(df)}")
df = df.dropna()
print(f"Number of samples after dropping: {len(df)}")
df.isna().sum(axis=0)

### Feature Scaling

Almost all machine learning models require features to be features to be on the same scale, this prevents the model from being biased towards features with larger scales.  
It's therefore good practice to always scale features before fitting a model.

In [ ]:
from ml_nb_code import feature_scaling
feature_scaling()

Most common method is **Z-score standardisation**, which transforms the data to have zero mean and unit variance.  
The formula for Z-score standardisation is:   
$X_{std} = \frac{X - \mu} {\sigma}$

In [ ]:
# Apply standardisation 
from sklearn.preprocessing import StandardScaler
import notebook_code as nc
df = nc.get_fs_data()

# Initialize the StandardScaler
scaler = StandardScaler()

# Fit the scaler to the data (i.e. calculate the mean and standard deviation)
scaler.fit(df[["x1", "x2"]])

# Transform the data
df[["x1_std", "x2_std"]] = scaler.transform(df[["x1", "x2"]])

print(f"Mean: {df['x1_std'].mean()}, Std: {df['x1_std'].std()}")
print(f"Mean: {df['x2_std'].mean()}, Std: {df['x2_std'].std()}")

# Plot the data
nc.vis_scaling(df)

### Categorial Features/Inputs

Machine learning models require numerical inputs, so categorial features need to be encoded.  

There are two types of categorial features:

1. **Ordinal**: Features with an inherent order, e.g. small, medium, large
2. **Nominal**: Features without an inherent order, e.g. red, green, blue

In [ ]:
# Example data
import notebook_code as nc
df = nc.load_cat_example()
df

#### Ordinal Feature Mapping
- Map the categories to numerical values, such that the numerical values reflect the order of the categories
- Additionally, the numerical values should be close to the actual difference between the categories
- This generally means that the mapping has to be defined manually
- For example, small=0, medium=1, large=2

In [ ]:
# Map the ordinal feature
size_mapping = {
    "M": 1,
    "L": 2,
    "XL": 3
}
df["size_encoded"] = df["size"].map(size_mapping)
df

### Nominal Feature Encoding
- Cannot use the same approach as for ordinal features, as there is no inherent order!
- For example setting green=1, red=2, blue=3 would imply that blue is larger than green
- This is commonly addressed by using one-hot encoding, which creates a new binary feature for each category

In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

# Initialize OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)
# Fit the encoder to the data, (i.e. determine the categories)
encoder.fit(df[["color"]])
# Get encoded feature column names
encoded_columns = encoder.get_feature_names_out(['color'])
# Transform the data
df[encoded_columns] = encoder.transform(df[["color"]])
df

### Label encoding
Similarly to features, ML models also require the target variable to be numerical. This is generally done using label encoding, i.e. assigning an integer value to each class.  
 The integer value assigned to the classes have no meaning, so can just enumerate the classes in the order they appear in the data. 

In [ ]:
# Load weather data
import notebook_code as nc
df = nc.load_weather_data_pre()
df.head()

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder
label_encoder = LabelEncoder()
# Fit and transform the data
df["target_encoded"] = label_encoder.fit_transform(df["target"])
df

In [ ]:
# Inverse transform
df["target_2"] = label_encoder.inverse_transform(df["target_encoded"])
df

### Hands-On Data Preprocessing
- Perform data preprocessing of the full weather dataset

In [ ]:
# Load data
import notebook_code as nc
df = nc.load_weather_data(with_missing_values=True, subset=True, features=["temperature", "humidity", "location_type"])
df.head()

In [ ]:
### Hands-on section
# 1) Handle missing data
# 2) Standardise numerical features
# 3) Encode categorial features
# 4) Encode target variable

### Your code here
# TODO: Hide the solution!
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder

# Drop rows with missing data
print(f"Number of samples before dropping: {len(df)}")
df = df.dropna()
print(f"Number of samples after dropping: {len(df)}")
df.isna().sum(axis=0)

# Standardise numerical features
numerical_features = ["temperature", "humidity"]
scaler = StandardScaler()
scaler.fit(df[numerical_features])
df[numerical_features] = scaler.transform(df[numerical_features])

# Encode categorial features
nominal_features = ["location_type"]
encoder = OneHotEncoder(sparse_output=False)
encoder.fit(df[nominal_features])
encoded_columns = encoder.get_feature_names_out(nominal_features)
df[encoded_columns] = encoder.transform(df[nominal_features])

# Encode target variable
label_encoder = LabelEncoder()
df["target_encoded"] = label_encoder.fit_transform(df["target"])

df.head()

## Model Fitting
In this notebook we will focus solely on decision tree models, however there are many other model types available.

List of most common ML models according to ChatGPT:
> The most commonly used supervised machine learning models are:
> 
> 1. **Linear Regression**: Used for predicting continuous values by modeling the relationship between input features and the target variable.
> 
> 2. **Logistic Regression**: Used for binary classification tasks by estimating the probability that an instance belongs to a particular class.
> 
> 3. **Decision Trees**: A versatile model that splits data into subsets based on feature values, used for both classification and regression tasks.
> 
> 4. **Random Forest**: An ensemble of decision trees that improves performance by averaging the predictions of multiple trees to reduce overfitting.
> 
> 5. **Support Vector Machines (SVM)**: A powerful model for classification tasks that finds the optimal hyperplane to separate different classes.
> 
> 6. **k-Nearest Neighbors (k-NN)**: A simple classification model that assigns labels based on the majority class of the k-nearest neighbors in the training data.
> 
> 7. **Naive Bayes**: A probabilistic classifier based on Bayes' theorem, assuming independence between features, often used for text classification.
> 
> 8. **Gradient Boosting Machines (GBM)**: An ensemble model that builds trees sequentially, with each new tree correcting errors made by the previous ones (e.g., XGBoost, LightGBM, CatBoost).
> 
> 9. **Neural Networks**: Particularly effective for complex tasks like image recognition and natural language processing, neural networks can model intricate patterns by learning from multiple layers of abstraction.
> 
> 10. **k-Nearest Neighbors (k-NN)**: A non-parametric model used for both classification and regression, which predicts the output based on the closest training examples in the feature space.
> 
> These models are widely used due to their effectiveness across various domains and their ability to handle different types of data and tasks.

The scikit-learn library has implementation for almost all of these, see the [documentation](https://scikit-learn.org/stable/user_guide.html) for more details.

### Decision Tree  
<img src="resources/imgs/ml_decision_tree_example.png">

### Decision Tree - Manual Fitting

In [ ]:
import man_decision_tree as mdt
mdt.run_man_decision_tree()

### Decision Tree - Fitting

In [ ]:
# Load the pre-processed data
import notebook_code as nc
df, features, label_encoder = nc.load_weather_data(subset=True, pre_process=True, features=["temperature", "humidity", "location_type"])
df = df[features + ["target_encoded"]]
df

In [ ]:
# Fit a decision tree classifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import accuracy_score
from matplotlib import pyplot as plt

# Create a Decision Tree classifier
clf = DecisionTreeClassifier()

# Train the model on the training data
clf.fit(df[features], df["target_encoded"])

# Get model predictions
y_pred = clf.predict(df[features])

# Calculate accuracy
accuracy = accuracy_score(df["target_encoded"], y_pred)
print(f"Accuracy: {accuracy:.2f}")

In [ ]:
# Visualize the decision tree
plt.figure(figsize=(16, 10))
plot_tree(clf, filled=True, impurity=False, feature_names=features, class_names=label_encoder.inverse_transform(clf.classes_))
plt.show()

Note: Left corresponds to True, Right corresponds to False

In [ ]:
### Plot decision boundary
nc.plot_decision_boundaries(df, clf, features, label_encoder)

### Overfitting and Underfitting

In [ ]:
import notebook_code as nc
nc.linear_regression_fitting_example()

## How to tell if a model is overfitting or underfitting? - Model Evaluation
- Evaluate model performance on unseen data

In [ ]:
import notebook_code as nc
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import accuracy_score
import pandas as pd

# Data loading
features = ["temperature", "humidity", "location_type"]
df = nc.load_weather_data(subset=True, features=features)

# Split into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(df[features],
                                                  df["target"],
                                                  test_size=0.2,
                                                  random_state=42)
train_df = pd.concat([X_train, y_train], axis=1)
val_df = pd.concat([X_val, y_val], axis=1)

print(f"Number of samples before splitting: {len(df)}")
print(f"Number of training samples: {len(train_df)}")
print(f"Number of validation samples: {len(val_df)}")

In [ ]:
### Pre-processing
# Standardise numerical features
numerical_features = ["temperature", "humidity"]
scaler = StandardScaler()
scaler.fit(train_df[numerical_features])
train_df[numerical_features] = scaler.transform(train_df[numerical_features])
val_df[numerical_features] = scaler.transform(val_df[numerical_features])

# Encode categorial features
nominal_features = ["location_type"]
encoder = OneHotEncoder(sparse_output=False)
encoder.fit(train_df[nominal_features])
encoded_columns = encoder.get_feature_names_out(nominal_features)
train_df[encoded_columns] = encoder.transform(train_df[nominal_features])
val_df[encoded_columns] = encoder.transform(val_df[nominal_features])
features = numerical_features + list(encoded_columns)

# Encode target variable
label_encoder = LabelEncoder()
train_df["target_encoded"] = label_encoder.fit_transform(train_df["target"])
val_df["target_encoded"] = label_encoder.transform(val_df["target"])

In [ ]:
# Create a Decision Tree classifier
clf = DecisionTreeClassifier()

# Train the model on the training data
clf.fit(train_df[features], train_df["target_encoded"])

# Get model predictions
train_y_pred = clf.predict(train_df[features])
val_y_pred = clf.predict(val_df[features])

# Calculate accuracy
train_accuracy = accuracy_score(train_df["target_encoded"], train_y_pred)
val_accuracy = accuracy_score(val_df["target_encoded"], val_y_pred)
print(f"Training Accuracy: {train_accuracy:.4f}")
print(f"Validation Accuracy: {val_accuracy:.4f}")

# Plot the decision boundary
nc.plot_decision_boundaries(train_df, clf, features, label_encoder, val_df=val_df, show_train=False)

### Holdout Method
- Split available (labelled) data into a training and validation set
- Generally split data 80% for training and 20% for validation
- Scikit-learn function: [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)

## How To Address Underfitting/Overfitting?

- Collect more data
- Data augmentation
- Feature engineering
- Feature selection
- Increase/Reduce model complexity
- Decrease/Increase applied regularization
- Explore different model types

## Hyperparameters
Parameters that are set before the training of the model that control the learning process.  
Unlike model parameters, hyperparameters are not learned during training, instead these are set by the user.  
They have a significant impact on the model performance and can be used to address overfitting/underfitting.    
  
They are generally model specific, for example the hyperparameters for a Decision Tree model are different from those of a Neural Network model.

Examples for a Decision Tree:
- Maximum depth of the tree
- Minimum number of samples required to split a node
- Minimum number of samples required at each leaf node
- For a full list see the Decision Tree [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html)

In [ ]:
### Hyperparameter Example
import notebook_code as nc
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import accuracy_score
import pandas as pd

# Data loading
features = ["temperature", "humidity", "location_type"]
df = nc.load_weather_data(subset=True, features=features)

# Split into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(df[features],
                                                  df["target"],
                                                  test_size=0.2,
                                                  random_state=42)
train_df = pd.concat([X_train, y_train], axis=1)
val_df = pd.concat([X_val, y_val], axis=1)

print(f"Number of samples before splitting: {len(df)}")
print(f"Number of training samples: {len(train_df)}")
print(f"Number of validation samples: {len(val_df)}")

# Run the pre-processing
train_df, val_df, features, label_encoder = nc.run_weather_preprocess(train_df, val_df)

# Create a Decision Tree classifier
clf = DecisionTreeClassifier(max_depth=4)

# Train the model on the training data
clf.fit(train_df[features], train_df["target_encoded"])

# Get model predictions
train_y_pred = clf.predict(train_df[features])
val_y_pred = clf.predict(val_df[features])

# Calculate accuracy
train_accuracy = accuracy_score(train_df["target_encoded"], train_y_pred)
val_accuracy = accuracy_score(val_df["target_encoded"], val_y_pred)
print(f"Training Accuracy: {train_accuracy:.4f}")
print(f"Validation Accuracy: {val_accuracy:.4f}")

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(16, 10))
plot_tree(clf, filled=True, impurity=False, feature_names=features, class_names=label_encoder.inverse_transform(clf.classes_))
plt.show()

In [ ]:
### Interactive Hyperparameter Example
### Task/Questions:
# - What do these hyperparameters do? See the documentation, https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html
# - How do they impact the model?
# - What are the differences and commonalities between the three hyperparameters?
# - What are some other hyperparameters that could be used?
# - What is the best hyperparameter combination?
import notebook_code as nc
nc.manual_hyperparam_tuning()

### Complete model evaluation strategy
- **Training dataset**: Used to train the model
- **Validation dataset**: Used to evaluate the model performance during hyperparameter tuning
- **Test dataset**: Used to evaluate the final model performance

<img src="resources/imgs/ml_train_val_test.png">

In [ ]:
### Variation in holdout runs
import notebook_code as nc
nc.holdout_variations()


### K-Fold Cross Validation
- Provides a more robust estimate of the model performance
- Gives indication on uncertainty in the model performance
- Also supported by scikit-learn: [cross_val_score](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html)

<img src="resources/imgs/ml_cross_val.png" width="1000"/>

In [ ]:
### Cross Validation Example
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
import notebook_code as nc

# Get the dataset
features = ["temperature", "humidity", "location_type"]
df, features, label_encoder = nc.load_weather_data(subset=True, features=features, pre_process=True)

# Create a Decision Tree classifier
clf = DecisionTreeClassifier()

# Use cross-validation to evaluate the model
cv_scores = cross_val_score(clf, df[features], df["target_encoded"], cv=5)

# Print the cross-validation scores
print(f"Cross-validation scores: {', '.join([f'{cur_score:.3f}' for cur_score in cv_scores])}")
print(f"Mean cross-validation score: {cv_scores.mean():.2f}")
print(f"Standard deviation of cross-validation scores: {cv_scores.std():.2f}")

## How to select the "best" hyperparameter values?

### Hyperparameter tuning
- Basic idea: Test different hyperparameter value combinations, evaluate their performance and select the best one
- Lots of different methods available, see the [Scikit-learn User Guide](https://scikit-learn.org/stable/modules/grid_search.html) for more details

##### Grid Search
Determines the best hyperparameters by exhaustively searching through a specified parameter grid   

##### Random Search
Determines the best hyperparameters by randomly sampling from a specified parameter grid

<img src="resources/imgs/ml_hyper_tuning.png" width="1000"/>

In [ ]:
### Grid Search Example
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
import notebook_code as nc

features = ["temperature", "humidity", "location_type"]
df, features, label_encoder = nc.load_weather_data(subset=True, features=features, pre_process=True)

# Create the classifier
clf = DecisionTreeClassifier(random_state=42)

# Define the parameter grid for the search
param_grid = {
    'max_depth': [2, 4, 8],
    'min_samples_split': [2, 5, 10, 20],
    'min_samples_leaf': [1, 5, 10, 20]
}

# Perform the grid search using cross-validation
grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, cv=5, n_jobs=-1, scoring='accuracy')
grid_search.fit(df[features], df["target_encoded"])

# Print the best parameters and the best score
print(f"Best parameters: {grid_search.best_params_}")

In [ ]:
# Create a dataframe with the results
results = []
for cur_param in param_grid.keys():
    results.append(pd.DataFrame(grid_search.cv_results_)[f"param_{cur_param}"])

results_df = pd.concat(results, axis=1)
results_df["mean_val_score"] = grid_search.cv_results_["mean_test_score"]
results_df["std_val_score"] = grid_search.cv_results_["std_test_score"]

In [ ]:
results_df.sort_values("mean_val_score", ascending=False).head(10)

## Roadmap

<img src="resources/imgs/ml_roadmap.png">

## Full Hands-On Example
- Apply the above concepts to a new problem
- Below are multiple options, if most of the content above was new for you then we recommend the first option.

Also feel free to any model type you like, not just Decision Trees.
For a list of supported models by scikit-learn see the [documentation](https://scikit-learn.org/stable/user_guide.html).  
The random forest model is a good choice if you want to try something new. It is an ensemble model that uses multiple decision trees to improve performance.

### Full Weather Dataset
The examples above only used a subset of the weather pattern classification dataset. See of you can apply the concepts to the full dataset and improve the model performance.  
[Dataset description](https://www.kaggle.com/datasets/nikhil7280/weather-type-classification)

In [ ]:
### Loading of the full weather dataset
import notebook_code as nc
df = nc.load_weather_data()

In [ ]:
df.head()

### Concrete Compressive Strength Dataset (Regression)
[Dataset description](https://archive.ics.uci.edu/dataset/165/concrete+compressive+strength)
 

In [ ]:
# Loading of the dataset
df = pd.read_csv("resources/concrete.csv")

In [ ]:
df.head()

### Heart Disease Dataset (Classification)
Predict the presence of heart disease based on various features.
[Dataset details](https://archive.ics.uci.edu/dataset/45/heart+disease)

In [ ]:
# Loading of the dataset
import notebook_code as nc
df = nc.load_heart_df() 

In [ ]:
df.head()

### New Zealand GM Prediction (Regression)
Use historical earthquakes from New Zealand to develop a ML-based GMM.

In [ ]:
# Loading of the dataset
df = pd.read_csv("resources/ground_motion_im_table_rotd50_flat.csv", dtype={"evid": str})